# Year Countdown
*This is a draft for what the "buggy narrative" option for the blog post might look like.*

We want to make a one year long timer. Specifically we want to make an object that can:
 * Record the time one year from when it is initialized.
 * Tell us how long until that point in time.
 
Lets start with a pretty buggy version of this code and then go through the various reasons why this code might break.

In [1]:
import datetime

In [2]:
class YearCountdown:
    def __init__(self):
        self.end = datetime.datetime.now()
        self.end = self.end.replace(year = self.end.year + 1)
        print(f"Set a countdown for {self.end}")
    def timeLeft(self):
        return self.end - datetime.datetime.now()

In [47]:
yc = YearCountdown()
print(yc.timeLeft())

Set a countdown for 2025-06-24 10:28:27.786208-04:00
8759:59:59.999519


Seems pretty good! We can call the object and get the correct cooldown. So, why is this wrong?
## Problem 1: Leap Years
What happens when we create a `YearCountdown` object on Feb 29th 2024? Feb 29 2025 isn't a valid date, so where does the end date fall? 

If we use a package `freezegun` to simulate this case, you can see that we get a runtime error! There is no Feb 29 2025, so the whole function fails.

## Solution 1: timedelta
Instead of setting the year value manually, we can instead use `timedelta` to increment one year safely.

In [48]:
from freezegun import freeze_time

with freeze_time("2024-02-29"):
    yc = YearCountdown()
    print(yc.timeLeft())

Set a countdown for 2025-02-28 00:00:00-05:00
8760:00:00


In [5]:
class YearCountdown:
    def __init__(self):
        self.end = datetime.datetime.now() + datetime.timedelta(days=365)
        print(f"Set a countdown for {self.end}")
    def timeLeft(self):
        return self.end - datetime.datetime.now()

In [32]:
with freeze_time("2024-02-29"):
    yc = YearCountdown()
    print(yc.timeLeft())

Set a countdown for 2025-02-28 19:00:00-05:00
in 1 year


## Issue 2: Leap Years (again)
While this works for the leap day itself, this actually manages to break even more days than it fixes! On leap years, the year is more than 365 days long, so our timedelta will be off by a day nearly 1/4 of the time. (notice how the countdown is set for the 18th, not the 19th).

## Solution 2: relativedelta
Datetime itself doesn't have a good solution to this problem, so we'll need to reach into another library. `dateutil` has an object called `relativedelta` allows us to add a year instead of 365 days.

NOTE: `relativedelta` has two sets of arguments labeled `year`, `month`, `day`...  and `years`, `months`, `days`... which act differently. The singular versions *set* the time and the plural versions *shift* the time. As a rule of thumb, you usually want the plural versions since the singular versions are handled by `.replace()`.

In [7]:
with freeze_time("2024-01-19"):
    yc = YearCountdown()
    print(yc.timeLeft())

Set a countdown for 2025-01-18 00:00:00
365 days, 0:00:00


In [8]:
from dateutil.relativedelta import relativedelta

class YearCountdown:
    def __init__(self):
        self.end = datetime.datetime.now() + relativedelta(years=1)
        print(f"Set a countdown for {self.end}")
    def timeLeft(self):
        return self.end - datetime.datetime.now()

In [49]:
with freeze_time("2024-01-19"):
    yc = YearCountdown()
    print(yc.timeLeft())

Set a countdown for 2025-01-19 00:00:00-05:00
8784:00:00


## Issue 3: DST
Although most times are exactly 365 or 366 days away from one year in the future, there are a few exceptions. Consider Mar 9 2024. In most of the US, that's right before the beginning of daylight savings time. However, Mar 9 2025 is *during* daylight savings time. Because of how the start and end date are set, daylight savings actually moves around from year to year. Between 9:00 on Mar 9 2024 and 9:00 the next year, there are 364 days and 23 hours.
## Solution 3: astimezone
The problem stems from the fact that by default all `datetime` objects in python are *naive* times, which means they don't have an associated timezone. A call to `.astimezone()` sets the timezones to whatever your system clock is using and 

In [10]:
with freeze_time("2024-03-09 09:00:00"):
    yc = YearCountdown()
    print(yc.timeLeft())

Set a countdown for 2025-03-09 09:00:00
365 days, 0:00:00


In [11]:
class YearCountdown:
    def __init__(self):
        self.end = datetime.datetime.now() + relativedelta(years=1)
        print(f"Set a countdown for {self.end}")
    def timeLeft(self):
        return self.end.astimezone() - datetime.datetime.now().astimezone()

In [50]:
with freeze_time("2024-03-09 09:00:00"):
    yc = YearCountdown()
    print(yc.timeLeft())

Set a countdown for 2025-03-09 09:00:00-04:00
8759:00:00


## Side Note: Missing Hours
The issue that we had with Feb 29 involved us finding a day that only exists in one year, how about an hour that only exists in one year? 2:30 on Mar 9 2025 doesn't exist because of daylight savings. After 1:59, clocks immediately skip to 3:00. If we create a countdown exactly a year before then, python's datetime library does something odd: when it resolves 2:30 into a timezone which doesn't have that time, it shifts it down exactly an hour to 1:30. This behavior isn't incorrect, but it's not obvious and can lead to strange results.

In [51]:
with freeze_time("2024-03-09 02:30:00"):
    yc = YearCountdown()
    print(yc.timeLeft())

Set a countdown for 2025-03-09 03:30:00-04:00
8760:00:00


## Issue 4: Changing timezones
Now that we have support for timezones in our program, we have to start thinking what might happen if you change timezones between when you call the initializer and when you call `timeLeft`. There isn't an obviously correct answer here, but there are two good candidates.

Lets say you call the constructor in timezone A on 9:00AM Jun 21 2024. You then move to timezone B and check your time left. The program can either tell you:
 1. The time until 9:00AM Jun 21 2025 in zone A
 2. The time until 9:00AM Jun 21 2025 in zone B

Our program currently gives you answer 2, so for the sake of demonstration, lets assume that we want to find answer 1.

NOTE: freezegun can't fully fake timezones, so you'll have to believe me on this one.

## Solution 4: move astimezone
This is a relatively fast fix, and it involves saving the timezone when you create the object instead of when you call `timeLeft`.

In [14]:
class YearCountdown:
    def __init__(self):
        self.end = datetime.datetime.now().astimezone() + relativedelta(years=1)
        print(f"Set a countdown for {self.end}")
    def timeLeft(self):
        return self.end - datetime.datetime.now().astimezone() 

# Bonus: other libraries
While Datetime works fine for simple date and time operations, you have hopefully seen that it is already becoming cumbersome to use in this relatively simple use case. I won't be going into a full pro/con list here of the alternatives here, but I can show you what the equivalent class would look like with a few of the other python date libraries.

Freezegun doesn't interface will all of these libraries correctly, so some of the tests may look broken. 

In [19]:
import arrow
class YearCountdown:
    def __init__(self):
        self.end = arrow.now().shift(years=1)
        print(f"Set a countdown for {self.end}")
    def timeLeft(self):
        return self.end - arrow.now()

In [37]:
import pendulum
class YearCountdown:
    def __init__(self):
        self.end = pendulum.now().add(years=1)
        print(f"Set a countdown for {self.end}")
    def timeLeft(self):
        return self.end.diff(pendulum.now())
# This prints weirldy, but calling .in_hours() or .in_seconds() on it gets it into a more recognizable form

In [46]:
import whenever
class YearCountdown:
    def __init__(self):
        self.end = whenever.LocalSystemDateTime.now() + whenever.years(1)
        self.end = self.end.as_offset()
        print(f"Set a countdown for {self.end}")
    def timeLeft(self):
        return self.end - whenever.LocalSystemDateTime.now()